<a href="https://colab.research.google.com/github/DionKimmm/2019_Summer_DL_Prof_Seok/blob/master/190724_GAN_Ver1_Generate_MNIST_image_using_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAN을 이용한 MNIST 숫자 생성

## 목차

* 필요한 라이브러리 불러오기
* 데이터 불러오기 & 하이퍼파라미터 정의
 * 비지도학습이므로 레이블이 필요없다.
* GAN 신경망 모델 구성
 * Generator, Discriminator 구성
* 손실함수 정의 및 최적화
 * Gan Mini, max 수식 이용 cost 정의
 * Generator, Discriminator 각각에 대한 최적화 함수 정의
* 훈련데이터를 이용한 학습
 * Session을 열고, Epoch 수를 결정
 * Generator, Discriminator 각각에 신경망 학습
* 테스트 데이터를 이용한 최종 식별 결과 확인
 * 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장

## 1. 필요한 라이브러리 불러오기

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

## 2. 데이터 불러오기 & 하이퍼 파라미터 정의

In [2]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

#옵션 설정
total_epoch = 20
batch_size = 100
learning_rate = 0.0002
# 신경망 레이어 구성 옵션
n_hidden = 256
n_input = 28 * 28
n_noise = 128 # 생성기의 입력값으로 사용할 노이즈의 크기


W0724 06:44:42.122270 140353598953344 deprecation.py:323] From <ipython-input-2-21d829c29332>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0724 06:44:42.125396 140353598953344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0724 06:44:42.126529 140353598953344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0724 06:44:42.371330 140353598953344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0724 06:44:42.373851 140353598953344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0724 06:44:42.421394 140353598953344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


## 3. 신경망 모델 구성

In [0]:
#신경망 모델 구성
# GAN 도 Unsupervised 학습이므로 Autoencoder 처럼 Y 를 사용 않함. 
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise]) # 노이즈 Z를 입력값으로 사용

# 생성기 신경망에 사용하는 변수들
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

# 판별기 신경망에 사용하는 변수들
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
# 판별기의 최종 결과값은 얼마나 진짜와 가깝냐를 판단하는 한 개의 스칼라 값
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

# 생성기(G) 신경망을 구성
def generator(noise_z):
      hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
      output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
      return output

# 판별기(D) 신경망을 구성
def discriminator(inputs):
      hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
      output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
      return output

# 랜덤한 노이즈(Z)
def get_noise(batch_size, n_noise):
      return np.random.normal(size=(batch_size, n_noise))
  
# 노이즈를 이용해 랜덤한 이미지를 생성합니다.
G = generator(Z)

# 노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구합니다.
D_gene = discriminator(G)

# 진짜 이미지를 이용해 판별한 값을 구합니다.
D_real = discriminator(X)

## 4. 손실함수 정의 및 최적화

In [0]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

loss_G = tf.reduce_mean(tf.log(D_gene))

D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

# 최적화 하려는 loss_D 와 loss_G 에 음수 부호를 붙여줍니다.
train_D= tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G= tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

## 5 & 6. 훈련 데이터를 GAN 신경망 모델 학습 및 테스트

In [7]:
#신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    noise = get_noise(batch_size, n_noise)

    # 판별기와 생성기 신경망을 각각 학습시킵니다.
    _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
    _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
             
  print('Epoch:', '%04d' % epoch,'D loss: {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))

  # 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
  if epoch == 0 or (epoch + 1) % 10 == 0:
    sample_size = 10
    noise = get_noise(sample_size, n_noise)
    samples = sess.run(G, feed_dict={Z: noise})

    fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
    for i in range(sample_size):
      ax[i].set_axis_off()
      ax[i].imshow(np.reshape(samples[i], (28, 28)))
    plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    plt.close(fig)
    print('최적화 완료!')

Epoch: 0000 D loss: -0.4711 G loss: -2.266
최적화 완료!
Epoch: 0001 D loss: -0.1709 G loss: -3.013
Epoch: 0002 D loss: -0.1251 G loss: -3.11
Epoch: 0003 D loss: -0.3534 G loss: -2.151
Epoch: 0004 D loss: -0.3395 G loss: -1.935
Epoch: 0005 D loss: -0.3777 G loss: -2.131
Epoch: 0006 D loss: -0.2498 G loss: -2.64
Epoch: 0007 D loss: -0.1585 G loss: -2.966
Epoch: 0008 D loss: -0.3544 G loss: -2.453
Epoch: 0009 D loss: -0.1833 G loss: -3.075
최적화 완료!
Epoch: 0010 D loss: -0.4902 G loss: -2.536
Epoch: 0011 D loss: -0.4113 G loss: -2.371
Epoch: 0012 D loss: -0.4204 G loss: -2.325
Epoch: 0013 D loss: -0.2808 G loss: -2.34
Epoch: 0014 D loss: -0.3657 G loss: -2.456
Epoch: 0015 D loss: -0.352 G loss: -2.658
Epoch: 0016 D loss: -0.4894 G loss: -2.374
Epoch: 0017 D loss: -0.4678 G loss: -2.156
Epoch: 0018 D loss: -0.2524 G loss: -2.767
Epoch: 0019 D loss: -0.4332 G loss: -2.383
최적화 완료!


In [0]:
for i in range(sample_size):
  ax[i].set_axis_off()
  ax[i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
  plt.show()